# Classification du Paludisme avec PyTorch\n\nCe notebook implémente 3 modèles CNN pour détecter le paludisme dans des images d'hématies:\n1. **CNN Simple** (from scratch)\n2. **VGG16** (fine-tuned)\n3. **ResNet50** (fine-tuned)

In [ ]:
import torch\nimport torch.nn as nn\nimport torch.optim as optim\nfrom torch.utils.data import Dataset, DataLoader\nfrom torchvision import transforms, models\n\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom PIL import Image\nimport os\nfrom tqdm import tqdm\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc\n\nprint(f\"PyTorch version: {torch.__version__}\")\n\nif torch.cuda.is_available():\n    device = torch.device('cuda')\n    print(f\"GPU disponible: {torch.cuda.get_device_name(0)}\")\nelif torch.backends.mps.is_available():\n    device = torch.device('mps')\n    print(\"GPU Apple Silicon (MPS) disponible\")\nelse:\n    device = torch.device('cpu')\n    print(\"Utilisation du CPU\")\n\nplt.style.use('seaborn-v0_8-darkgrid')

## 1. Configuration

In [ ]:
taille_image = 64\ntaille_batch = 32\nnombre_epochs = 30\ntaux_apprentissage = 0.001\npatience_early_stopping = 5\n\nclasses = ['Parasitee', 'Non_parasitee']\nnombre_classes = 2\n\nchemin_donnees = './data/malaria_hematie_dataset'\nchemin_resultats = './resultats'\nos.makedirs(chemin_resultats, exist_ok=True)\n\nprint(f\"Configuration:\")\nprint(f\"  - Taille image: {taille_image}x{taille_image}\")\nprint(f\"  - Batch size: {taille_batch}\")\nprint(f\"  - Epochs max: {nombre_epochs}\")\nprint(f\"  - Learning rate: {taux_apprentissage}\")\nprint(f\"  - Device: {device}\")

## 2. Chargement et Préparation des Données

In [ ]:
class MalariaDataset(Dataset):\n    \n    def __init__(self, chemins_images, labels, transform=None):\n        self.chemins_images = chemins_images\n        self.labels = labels\n        self.transform = transform\n    \n    def __len__(self):\n        return len(self.chemins_images)\n    \n    def __getitem__(self, index):\n        chemin_image = self.chemins_images[index]\n        image = Image.open(chemin_image).convert('RGB')\n        label = self.labels[index]\n        \n        if self.transform:\n            image = self.transform(image)\n        \n        return image, label

In [ ]:
print(\"Chargement des images...\")\n\nchemins_images_liste = []\nlabels_liste = []\n\nfor index_classe, nom_classe in enumerate(classes):\n    chemin_classe = os.path.join(chemin_donnees, nom_classe)\n    \n    if not os.path.exists(chemin_classe):\n        print(f\"Attention: {chemin_classe} n'existe pas\")\n        continue\n    \n    for nom_image in os.listdir(chemin_classe):\n        chemin_complet = os.path.join(chemin_classe, nom_image)\n        chemins_images_liste.append(chemin_complet)\n        labels_liste.append(index_classe)\n\nprint(f\"Total d'images chargées: {len(chemins_images_liste)}\")\nfor index_classe, nom_classe in enumerate(classes):\n    nombre_images_classe = labels_liste.count(index_classe)\n    print(f\"  - {nom_classe}: {nombre_images_classe} images\")

In [ ]:
chemins_train, chemins_temp, labels_train, labels_temp = train_test_split(\n    chemins_images_liste, labels_liste, test_size=0.3, random_state=42, stratify=labels_liste\n)\n\nchemins_val, chemins_test, labels_val, labels_test = train_test_split(\n    chemins_temp, labels_temp, test_size=0.5, random_state=42, stratify=labels_temp\n)\n\nprint(f\"Répartition des données:\")\nprint(f\"  - Train: {len(chemins_train)} images\")\nprint(f\"  - Validation: {len(chemins_val)} images\")\nprint(f\"  - Test: {len(chemins_test)} images\")

In [ ]:
transform_train = transforms.Compose([\n    transforms.Resize((taille_image, taille_image)),\n    transforms.RandomRotation(20),\n    transforms.RandomHorizontalFlip(),\n    transforms.RandomVerticalFlip(),\n    transforms.ToTensor(),\n    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])\n])\n\ntransform_test = transforms.Compose([\n    transforms.Resize((taille_image, taille_image)),\n    transforms.ToTensor(),\n    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])\n])\n\ndataset_train = MalariaDataset(chemins_train, labels_train, transform=transform_train)\ndataset_val = MalariaDataset(chemins_val, labels_val, transform=transform_test)\ndataset_test = MalariaDataset(chemins_test, labels_test, transform=transform_test)\n\ndataloader_train = DataLoader(dataset_train, batch_size=taille_batch, shuffle=True)\ndataloader_val = DataLoader(dataset_val, batch_size=taille_batch, shuffle=False)\ndataloader_test = DataLoader(dataset_test, batch_size=taille_batch, shuffle=False)\n\nprint(\"DataLoaders créés avec succès\")

### Visualisation des Images

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(15, 6))\n\nfor index_image in range(10):\n    row = index_image // 5\n    col = index_image % 5\n    \n    image = Image.open(chemins_train[index_image])\n    label_index = labels_train[index_image]\n    \n    axes[row, col].imshow(image)\n    axes[row, col].set_title(f\"{classes[label_index]}\", fontsize=10)\n    axes[row, col].axis('off')\n\nplt.tight_layout()\nplt.savefig(f'{chemin_resultats}/exemples_images.png', dpi=150, bbox_inches='tight')\nplt.show()